In [185]:
import numpy as np
from PIL import Image
from scipy.ndimage import gaussian_filter

# Charger et convertir l'image en niveaux de gris
img1 = Image.open('data/elephant.jpg').convert('L')
img2 = Image.open('data/dent.jpg').convert('L')

# Définir une taille commune
new_size = (256, 256)

# Redimensionner les images à la taille définie
img1 = img1.resize(new_size)
img2 = img2.resize(new_size)

# Convertir en tableaux numpy
img1_np = np.array(img1)/255  # Normaliser pour que les valeurs soient dans [0, 1]
img2_np = np.array(img2)/255


In [ ]:
#Image.fromarray(v.astype('uint8')) 

In [186]:
# Initialisation des vecteurs de probabilités
v = np.ones_like(img1_np)
w = np.ones_like(img2_np)

# Définir les paramètres du noyau de chaleur (heat kernel)
gamma = 1  # Paramètre de régularisation


In [187]:
# Fonction de convolution avec un noyau gaussien
def apply_heat_kernel(image, sigma):
    return gaussian_filter(image, sigma=sigma) #renvoie le tableau

# Paramètre du noyau de diffusion (dépend du paramètre de régularisation γ)
sigma = np.sqrt(gamma / 2)


In [197]:
# Fonction pour les itérations de Sinkhorn
def sinkhorn_iterations(mu0, mu1, iterations=100):
    v = np.ones_like(mu0)
    w = np.ones_like(mu1)
    
    for i in range(iterations):
        v = mu0 / apply_heat_kernel(w, sigma)
        w = mu1 / apply_heat_kernel(v, sigma)
    
    return v, w

# Appliquer les itérations
mu0_flat = img1_np.ravel()
mu1_flat = img2_np.ravel()
v, w = sinkhorn_iterations(mu0_flat, mu1_flat)


/var/folders/nk/rqbxbzbs6cx3djvmyg89hzhh0000gn/T/ipykernel_3079/1755996212.py:8: RuntimeWarning: divide by zero encountered in divide
  w = mu1 / apply_heat_kernel(v, sigma)
/var/folders/nk/rqbxbzbs6cx3djvmyg89hzhh0000gn/T/ipykernel_3079/1755996212.py:8: RuntimeWarning: overflow encountered in divide
  w = mu1 / apply_heat_kernel(v, sigma)


In [198]:
#Image.fromarray(v.astype('uint8'))  

In [199]:
def kl_divergence(mu0, mu1, v, w):
    return gamma * (np.sum(mu0 * np.log(v)) + np.sum(mu1 * np.log(w)))

# Calculer la divergence KL
wasserstein_distance = kl_divergence(mu0_flat, mu1_flat, v, w)
print(f"Convolutional Wasserstein Distance: {wasserstein_distance}")


Convolutional Wasserstein Distance: nan


/var/folders/nk/rqbxbzbs6cx3djvmyg89hzhh0000gn/T/ipykernel_3079/2282589170.py:2: RuntimeWarning: divide by zero encountered in log
  return gamma * (np.sum(mu0 * np.log(v)) + np.sum(mu1 * np.log(w)))
/var/folders/nk/rqbxbzbs6cx3djvmyg89hzhh0000gn/T/ipykernel_3079/2282589170.py:2: RuntimeWarning: invalid value encountered in multiply
  return gamma * (np.sum(mu0 * np.log(v)) + np.sum(mu1 * np.log(w)))
